In [1]:
import pandas as pd
import json
import ast

In [2]:
amazonnews_dataframe = pd.DataFrame(json.load(open('./Data/investnewsamazon.json')))
applenews_dataframe = pd.DataFrame(json.load(open('./Data/investnewsapple.json')))
googlenews_dataframe = pd.DataFrame(json.load(open('./Data/investnewsgoogle.json')))
metanews_dataframe = pd.DataFrame(json.load(open('./Data/investnewsmeta.json'))+json.load(open('./Data/investnewsmeta2.json')))
msftnews_dataframe = pd.DataFrame(json.load(open('./Data/investnewsmsft.json')))
nvidianews_dataframe = pd.DataFrame(json.load(open('./Data/investnewsnvidia.json')))
samsungnews_dataframe = pd.DataFrame(json.load(open('./Data/investnewssamsung.json')))
tencentnews_dataframe = pd.DataFrame(json.load(open('./Data/investnewstencent.json')))
teslanews_dataframe = pd.DataFrame(json.load(open('./Data/investnewstesla.json'))+json.load(open('./Data/investnewstesla2.json')))
tsmcnews_dataframe = pd.DataFrame(json.load(open('./Data/investnewstsmc.json')))


In [3]:
list_of_news_dataframes = [amazonnews_dataframe,
                            applenews_dataframe,
                            googlenews_dataframe,
                            metanews_dataframe,
                            msftnews_dataframe,
                            nvidianews_dataframe,
                            samsungnews_dataframe,
                            tencentnews_dataframe,
                            teslanews_dataframe,
                            tsmcnews_dataframe
                            ]

In [4]:
list_of_news_sources = []

for data in list_of_news_dataframes:
    for source in list(data["Article Source"].value_counts().index):
        list_of_news_sources.append(source)

list_of_news_sources = list(set(list_of_news_sources))

In [5]:
dict_of_news_sources = {'https://i-invdn-com.investing.com/news/providers/3e057c7f384e820886ff7ca52594bb7b.png':"TipRanks",
                        'https://i-invdn-com.investing.com/news/providers/eafe6a3d95927d68b2173cda2ef9dc1c.png':"StockNews",
                        'https://i-invdn-com.investing.com/news/providers/559698bc836be4174c42841b3c473303.png':"Cointelegraph",
                        'https://i-invdn-com.investing.com/news/providers/05f392742320643916d28dbf568b3e6d.png':"Coin_Edition",
                        'https://i-invdn-com.investing.com/news/providers/investing-new.png':"Investing.com",
                        'https://i-invdn-com.investing.com/news/providers/Reuters.png':"Reuters",
                        'https://i-invdn-com.investing.com/news/providers/bb01da8976e117e1320d62e0a2924827.png':"DailyCoin",
                        'https://i-invdn-com.investing.com/logos/cryptovest.png':"cryptovest",
                        'https://i-invdn-com.investing.com/news/providers/f669b6220c6f890e37e3e2eb71b9c82f.png':"FXstreet",
                        'https://i-invdn-com.investing.com/news/providers/a3de4bfb9ab7d4545a5ee8ca37b74267.png':"RealVision",
                        'https://i-invdn-com.investing.com/news/providers/05863679414a61354884428366cdf72d.jpg':"U.Today",
                        'https://i-invdn-com.investing.com/logos/Bloomberg_new.png':"Bloomberg",
                        'https://i-invdn-com.investing.com/news/providers/baa4350e859afe7cbe8cd35e7c6cc3d5.jpg':'BTC_Peers',
                        'https://i-invdn-com.investing.com/logos/Business-Insider-small.png':'Business_Insider',
                        'https://i-invdn-com.investing.com/news/providers/e28d7f12114c504773c172de3c698b61.png':"MoneyShow",
}

In [6]:
def text_preprocessor (text_list):
    if len(text_list) > 0:
        if text_list[0][0] =="(" and text_list[0][len(text_list[0])-1] == ")":
            text_list.pop(0)
        if text_list[0][0:3] == "By " and len(text_list[0])<=100:
            text_list.pop(0)
    return text_list


import string
string.punctuation

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in '!"#&\'()*+/:;<=>?@[\\]^_`{|}~'])
    
    return punctuationfree
#storing the punctuation free text

In [7]:
#Text Pre-Processing
def news_dataframe_preprocessing(dataframe):

    dataframe.replace({"Article Source": dict_of_news_sources}, inplace = True)

    dataframe['Date'] = dataframe['Date'].astype(str).str.extract('(\w{3}\s\d{2},\s\d{4}\s\d{2}:\d{2}\w{2})').fillna('')

    dataframe['Date'] = pd.to_datetime(dataframe['Date'])

    print(f"Original Dataframe Shape: {dataframe.shape}")

    dataframe.drop_duplicates(inplace=True,subset=['Date','Article Link'])

    print(f"Dataframe Shape after Removing Duplicates: {dataframe.shape}")

    dataframe = dataframe.loc[(dataframe['Date'] >= '2016-09-30')
                            & (dataframe['Date'] <= '2022-09-30')].copy()

    print(f"Dataframe Shape after Date Filtering: {dataframe.shape}")

    dataframe['Text'] = dataframe['Text'].map(text_preprocessor)

    dataframe['Text'] = [','.join(map(str, l)) for l in dataframe['Text']]

    dataframe['Text'] = dataframe['Text'].replace(r'\n',' ', regex=True) 

    dataframe["Cleaned_Text"] = dataframe['Text'].replace(r'(^.{0,20}Investing.com.?\–?)|(^.{0,20}Investing.?\–?)|(^.{0,20}\(Bloomberg.{0,20}\)\s\-+)|(^.{0,130}\(.{0,20}Reuters.{0,20}\)\s\-{1,2})','', regex=True) 

    dataframe["Cleaned_Text"] = dataframe["Cleaned_Text"].replace(r'^.{0,5}Stocks\sin\sfocus(.{0,40}day:|.{0,60}Please\srefresh\sfor\supdates)','', regex=True) 

    dataframe["Cleaned_Text"] = dataframe["Cleaned_Text"].replace(r'^.{0,5}Here.{0,100}(markets|market|radar)(\w{2,15}\.\W|\d{1,2}\:)\s?(Please\srefresh\sfor\supdates\.)?','', regex=True) 

    dataframe["Cleaned_Text"] = dataframe["Cleaned_Text"].replace(r'^Terms of Trade is a daily newsletter that untangles a world embroiled in trade wars.','', regex=True)

    dataframe["Cleaned_Text"] = dataframe["Cleaned_Text"].replace(r'-',' ', regex=True)

    dataframe["Cleaned_Text"] = dataframe["Cleaned_Text"].replace(r'\bo\b',' ', regex=True)    

    dataframe["Cleaned_Text"] = dataframe["Cleaned_Text"].replace(r'\W\W+',' ', regex=True)

    dataframe["Cleaned_Text"] = dataframe["Cleaned_Text"].replace(r'^.$','', regex=True)

    dataframe["Cleaned_Text"] = dataframe["Cleaned_Text"].apply(lambda x:remove_punctuation(x))

    dataframe["Cleaned_Text"]= dataframe["Cleaned_Text"].apply(lambda x: x.lower())

    return dataframe


In [8]:
amazonnews_dataframe_cleaned = news_dataframe_preprocessing(amazonnews_dataframe)
applenews_dataframe_cleaned = news_dataframe_preprocessing(applenews_dataframe)
googlenews_dataframe_cleaned = news_dataframe_preprocessing(googlenews_dataframe)
metanews_dataframe_cleaned = news_dataframe_preprocessing(metanews_dataframe)
msftnews_dataframe_cleaned = news_dataframe_preprocessing(msftnews_dataframe)
nvidianews_dataframe_cleaned = news_dataframe_preprocessing(nvidianews_dataframe)
samsungnews_dataframe_cleaned = news_dataframe_preprocessing(samsungnews_dataframe)
tencentnews_dataframe_cleaned = news_dataframe_preprocessing(tencentnews_dataframe)
teslanews_dataframe_cleaned = news_dataframe_preprocessing(teslanews_dataframe)
tsmcnews_dataframe_cleaned = news_dataframe_preprocessing(tsmcnews_dataframe)


Original Dataframe Shape: (13201, 6)
Dataframe Shape after Removing Duplicates: (13201, 6)
Dataframe Shape after Date Filtering: (12066, 6)
Original Dataframe Shape: (14960, 6)
Dataframe Shape after Removing Duplicates: (14960, 6)
Dataframe Shape after Date Filtering: (12619, 6)
Original Dataframe Shape: (12790, 6)
Dataframe Shape after Removing Duplicates: (12790, 6)
Dataframe Shape after Date Filtering: (10914, 6)
Original Dataframe Shape: (16339, 6)
Dataframe Shape after Removing Duplicates: (16331, 6)
Dataframe Shape after Date Filtering: (14464, 6)
Original Dataframe Shape: (7609, 6)
Dataframe Shape after Removing Duplicates: (7609, 6)
Dataframe Shape after Date Filtering: (6274, 6)
Original Dataframe Shape: (1886, 6)
Dataframe Shape after Removing Duplicates: (1886, 6)
Dataframe Shape after Date Filtering: (1788, 6)
Original Dataframe Shape: (2621, 6)
Dataframe Shape after Removing Duplicates: (2621, 6)
Dataframe Shape after Date Filtering: (2597, 6)
Original Dataframe Shape: (63

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [10]:
import torch

def sentiments(input):
    if len(input)>2:
        inputs = tokenizer(input, padding = True, truncation = True, return_tensors='pt')
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        return (predictions[0].tolist())

from textblob import TextBlob

def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

In [11]:
#Sentiment Scores
def news_dataframe_sentiment_transformation(dataframe):
    dataframe["Title Sentiment"] = dataframe["Title"].apply(lambda x: sentiments(x))

    dataframe["Title Positive"] = dataframe["Title Sentiment"].str[0].astype(float)
    dataframe["Title Negative"] = dataframe["Title Sentiment"].str[1].astype(float)
    dataframe["Title Neutral"] = dataframe["Title Sentiment"].str[2].astype(float)

    dataframe["Text Sentiment"] = dataframe["Cleaned_Text"].apply(lambda x: sentiments(x))

    dataframe["Text Positive"] = dataframe["Text Sentiment"].str[0].astype(float)
    dataframe["Text Negative"] = dataframe["Text Sentiment"].str[1].astype(float)
    dataframe["Text Neutral"] = dataframe["Text Sentiment"].str[2].astype(float)

    dataframe.drop(['Title Sentiment', 'Text Sentiment'], axis=1, inplace = True)

    dataframe["Title Subjectivity"] = dataframe["Title"].apply(getSubjectivity)
    dataframe["Text Subjectivity"] = dataframe["Cleaned_Text"].apply(getSubjectivity)

    return(dataframe)

In [12]:
list_of_news_dataframes_cleaned = [amazonnews_dataframe_cleaned,
                                    applenews_dataframe_cleaned,
                                    googlenews_dataframe_cleaned,
                                    metanews_dataframe_cleaned,
                                    msftnews_dataframe_cleaned,
                                    nvidianews_dataframe_cleaned,
                                    samsungnews_dataframe_cleaned,
                                    tencentnews_dataframe_cleaned,
                                    teslanews_dataframe_cleaned,
                                    tsmcnews_dataframe_cleaned
                            ]

In [13]:
i = 0
for dataframe in list_of_news_dataframes_cleaned:
    news_dataframe_sentiment_transformation(dataframe)
    i += 1
    print(f"Number of Dataframe Done:{i}")

Number of Dataframe Done:1
Number of Dataframe Done:2
Number of Dataframe Done:3
Number of Dataframe Done:4
Number of Dataframe Done:5
Number of Dataframe Done:6
Number of Dataframe Done:7
Number of Dataframe Done:8
Number of Dataframe Done:9
Number of Dataframe Done:10


In [15]:

amazonnews_dataframe_cleaned.to_csv('amazonnews.csv', index=False)
applenews_dataframe_cleaned.to_csv('applenews.csv', index=False)
googlenews_dataframe_cleaned.to_csv('googlenews.csv', index=False)
metanews_dataframe_cleaned.to_csv('metanews.csv', index=False)
msftnews_dataframe_cleaned.to_csv('msftnews.csv', index=False)
nvidianews_dataframe_cleaned.to_csv('nvidianews.csv', index=False)
samsungnews_dataframe_cleaned.to_csv('samsungnews.csv', index=False)
tencentnews_dataframe_cleaned.to_csv('tencentnews.csv', index=False)
teslanews_dataframe_cleaned.to_csv('teslanews.csv', index=False)
tsmcnews_dataframe_cleaned.to_csv('tsmcnews.csv', index=False)